In [12]:
import sys
import logging
import dataclasses
import pandas as pd
import numpy as np
import logging
sys.path.append('..')

import csv

In [13]:
from rnacloud_genome_reference.common.gtf import GTFHandler, SpliceJunctionPosition
from rnacloud_genome_reference.common.gnomad import GenomicRegion, query_genomic_regions, GenomicRegionQuerier

In [14]:
logging.basicConfig(level=logging.DEBUG)

In [15]:
pc = pd.read_csv("../temp/clinically_significant_protein_coding_genes.tsv", sep="\t", low_memory=False)

In [16]:
gtf_handler = GTFHandler("../tests/fixtures/GCF_000001405.40_GRCh38.p14_genomic.sorted.gtf.gz")

In [17]:
x = []

In [18]:



def convert_sj_positions_to_genomic_region(sj_position: SpliceJunctionPosition) -> GenomicRegion:
    return GenomicRegion(
        chrom='11',
        start=sj_position.pos,
        stop=sj_position.pos
    )


In [20]:
with open("../temp/clinically_significant_protein_coding_genes.tsv", 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')

    for row in reader:
        if row['entrez_gene_id'] == '9636':
            a = gtf_handler.obtain_sj_positions(
                chrom=row['chrom_refseq'],
                start=int(row['start']),
                end=int(row['end']),
                entrez_gene_id=int(row['entrez_gene_id'])
            )

            genomic_regions = [convert_sj_positions_to_genomic_region(sj) for sj in a]

            print(genomic_regions)

            gnomad_frequency = query_genomic_regions(genomic_regions)

            break



INFO:rnacloud_genome_reference.common.gtf:Fetching transcript for Entrez Gene ID: 9636 in primary region NC_000001.11:1013497-1014540
INFO:rnacloud_genome_reference.common.gtf:Obtaining transcript for Entrez Gene ID: 9636 at location NC_000001.11:1013497-1014540 (MANE: True)
DEBUG:rnacloud_genome_reference.common.gtf:Obtained transcript: NM_005101.4
INFO:rnacloud_genome_reference.common.gtf:Obtaining exons for transcript: NM_005101.4 at location NC_000001.11:1013497-1014540
DEBUG:rnacloud_genome_reference.common.gtf:Obtained Exons: [Exon(chromosome='NC_000001.11', start=1013497, end=1013576, strand='+', sequence=None, exon_no=1), Exon(chromosome='NC_000001.11', start=1013984, end=1014540, strand='+', sequence=None, exon_no=2)]
INFO:rnacloud_genome_reference.common.gnomad:Initialized GenomicRegionQuerier with endpoint: https://gnomad.broadinstitute.org/api
INFO:rnacloud_genome_reference.common.gnomad:Querying 4 genomic regions
DEBUG:rnacloud_genome_reference.common.gnomad:Adding region 

[GenomicRegion(chrom='11', start=1013577, stop=1013577, id=None), GenomicRegion(chrom='11', start=1013578, stop=1013578, id=None), GenomicRegion(chrom='11', start=1013982, stop=1013982, id=None), GenomicRegion(chrom='11', start=1013983, stop=1013983, id=None)]


DEBUG:urllib3.connectionpool:https://gnomad.broadinstitute.org:443 "POST /api HTTP/1.1" 400 83
INFO:gql.transport.requests:<<< {"errors":[{"message":"Query is too expensive (44). Maximum allowed cost is 25."}]}
ERROR:rnacloud_genome_reference.common.gnomad:Failed to execute GraphQL query: {'message': 'Query is too expensive (44). Maximum allowed cost is 25.'}


TransportQueryError: {'message': 'Query is too expensive (44). Maximum allowed cost is 25.'}